In [ ]:
!pip install fasttext transformers sacremoses memory-profiler
!pip install -U kaleido
from memory_profiler import memory_usage
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import StratifiedKFold
from google.colab import drive
import pandas as pd
import plotly.express as px
import os
import json
import spacy
import re
import pickle
import sklearn
from pathlib import Path
import time
import fasttext
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import numpy as np
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score  
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from transformers import AutoTokenizer, AutoModel
from copy import deepcopy
import warnings
import json
from torch.utils.tensorboard import SummaryWriter
warnings.filterwarnings("ignore")

tqdm.pandas()

## Description
This notebook presents the results of classification (pos/neg) of the reviews from the Booking.com dataset.  The classification was performed using MLP, Convolution, LSTM and embedding models trained with different techniques on multiple language corporas.

In [ ]:
drive_path = '/content/drive/'
BASE_DIR = drive_path + 'MyDrive/ColabNotebooks/studia/L3'
drive.mount(drive_path, force_remount=True)
log_dir = BASE_DIR + '/logs/'
FIGURES_DIR = BASE_DIR + "/figures"

os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir)

booking_path = os.path.join(BASE_DIR, "booking.txt")
dataframes_path = os.path.join(BASE_DIR, "dataframes")

EMBEDDING_FILES = [
    (os.path.join(dataframes_path, "clean_inputs_clean_booking_cbow_sampled.tar.gz"), "Booking CBOW"),
    (os.path.join(dataframes_path, "clean_inputs_clean_booking_skipgram_sampled.tar.gz"), "Booking Skip-Gram"),
    (os.path.join(dataframes_path, "clean_inputs_clean_oscar_cbow_sampled.tar.gz"), "Oscar CBOW"),
    (os.path.join(dataframes_path, "clean_inputs_clean_oscar_skipgram_sampled.tar.gz"), "Oscar Skip-Gram"),
    # (os.path.join(dataframes_path, "clean_herbert.pkl"), "HerBERT"), TODO: sample
    (os.path.join(dataframes_path, "clean_kgr10_sampled.tar.gz"), "KGR10 CBOW"),
]

Mounted at /content/drive/


In [ ]:
def get_training_data(df, column, max_samples):
    df = df[df[column].notnull()].sample(max_samples, random_state=1)
    return np.array(df[column].values.tolist())

def load_embeddings(path, max_samples_per_class=100):
    df = pd.read_pickle(path)
    x_res = []
    y_res = []
    for y_val, col in [(1, "vec_positive"), (0, "vec_negative")]:
        x = get_training_data(df, col, max_samples_per_class)
        x_res.append(x)
        y_res.append(np.full((x.shape[0], 1), y_val))
    
    return np.concatenate(x_res, axis=0), np.concatenate(y_res, axis=0)

## Model klasyfikacji

In [ ]:
class training_set(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.from_numpy(X).float()                          
        self.Y = torch.from_numpy(Y).float()                     

    def __len__(self):
        return len(self.X)                   

    def __getitem__(self, idx):
      return [self.X[idx], self.Y[idx]] 

def to_dl(x, y):
  train_data = training_set(x, y)
  return torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=32)

In [ ]:
class BookingClassifier(sklearn.base.BaseEstimator, sklearn.base.ClassifierMixin):
    def __init__(self, **kwargs):
        self.model = ...

    def init_model(self):
        pass

    def fit(self, X, y):
        # print("FIT")
        self.init_model()
        train_dl = to_dl(X, y)
        optimiser = optim.Adam(self.model.parameters())
        loss_fn = torch.nn.BCELoss()
        bar = tqdm(range(self.epochs))
        # print("le")
        for epoch in bar:
            self.model.train()
            for X_batch, y_batch in train_dl:
                X_batch = self.adjust(X_batch)
                y_pred = self.model(X_batch.cuda())
                loss = loss_fn(y_pred, y_batch.cuda())

                loss.backward()
                optimiser.step()
                optimiser.zero_grad() 
                bar.set_description(f"Loss: {loss:.4f}")
        return self

    def predict(self, X):
        return torch.round(self.model(self.adjust(torch.from_numpy(X).float().cuda())).detach().cpu()).numpy()

    def predict_proba(self, X):
        return self.model(self.adjust(torch.from_numpy(X).float().cuda())).detach().cpu().numpy()

    def adjust(self, X):
        return X

In [ ]:
class MLP(BookingClassifier):
    def __init__(self, epochs=20, in_dim=100, hid_dim=50, layers=2, **kwargs): 
        super(MLP, self).__init__(**kwargs)
        self.hid_dim = hid_dim
        self.layers = layers
        self.epochs = epochs
        self.in_dim = in_dim

    def init_model(self):
        self.model = nn.Sequential()
        curr_in = self.in_dim
        for i in range(self.layers):
            self.model.add_module(f"lin_{i}", torch.nn.Linear(curr_in, self.hid_dim)) 
            self.model.add_module(f"lin_{i}_act" , torch.nn.ReLU())
            curr_in = self.hid_dim

        self.model.add_module("lin_last", torch.nn.Linear(curr_in, 1)) 
        self.model.add_module("last_sigm" , torch.nn.Sigmoid())
        self.model.cuda()
    
    def adjust(self, X):
        return torch.mean(X, dim=1).squeeze()
    
    def fit(self, X, y):
        return super().fit(X, y)

In [ ]:
class LSTMReshape(nn.Module):
   
    def forward(self,x):
        out, hidden = x
        return out[: ,-1]


class RNN(BookingClassifier):
    def __init__(self, epochs=20, in_dim=100, hid_dim=50, layers=2, lin_hid=128, bi=True, dropout=0.1, **kwargs): 
        super(RNN, self).__init__(**kwargs)
        self.hid_dim = hid_dim
        self.layers = layers
        self.epochs = epochs
        self.in_dim = in_dim
        self.lin_hid = lin_hid
        self.bi = bi
        self.dropout = dropout

    def init_model(self):
        self.model = nn.Sequential()
        self.model.add_module("lstm", torch.nn.LSTM(batch_first=True, num_layers=self.layers, bidirectional=self.bi, dropout=self.dropout, input_size=self.in_dim, hidden_size=self.hid_dim))
        self.model.add_module("reshape", LSTMReshape())
        self.model.add_module("lstm_act", torch.nn.Tanh())
        # self.model.add_module("lin_pre_last", torch.nn.LazyLinear(self.lin_hid)) 
        # self.model.add_module("lin_pre_last_act", torch.nn.ReLU()) 
        self.model.add_module("lin_last", torch.nn.LazyLinear(1))  
        self.model.add_module("last_sigm" , torch.nn.Sigmoid())
        self.model.cuda()

    def fit(self, X, y):
        return super().fit(X, y)

In [ ]:
class Conv(BookingClassifier):
    def __init__(self, epochs=20, in_dim=100, hid_ch=50, layers=2, kernel_size=2, lin_hid=128, **kwargs): 
        super(Conv, self).__init__(**kwargs)
        self.hid_ch = hid_ch
        self.layers = layers
        self.epochs = epochs
        self.in_dim = in_dim
        self.kernel_size = kernel_size
        self.lin_hid = lin_hid

    def init_model(self):
        self.model = nn.Sequential()
        curr_in = self.in_dim
        curr_ch = self.in_dim
        for i in range(self.layers):
            self.model.add_module(f"conv_{i}", torch.nn.Conv1d(curr_in, self.hid_ch, self.kernel_size)) 
            self.model.add_module(f"conv_{i}_act" , torch.nn.ReLU())
            curr_in = self.hid_ch

        self.model.add_module("pool", torch.nn.MaxPool1d(self.kernel_size)) 
        self.model.add_module("flatten", torch.nn.Flatten()) 
        # self.model.add_module("lin_pre_last", torch.nn.LazyLinear(self.lin_hid)) 
        # self.model.add_module("lin_pre_last_act", torch.nn.ReLU()) 
        self.model.add_module("lin_last", torch.nn.LazyLinear(1))  
        self.model.add_module("last_sigm" , torch.nn.Sigmoid())
        self.model.cuda()
    
    def adjust(self, X):
        return X.permute(0,2,1)
    
    def fit(self, X, y):
        return super().fit(X, y)

## Uczenie i wyniki

In [ ]:
# X, y = load_embeddings(EMBEDDING_FILES[3][0], max_samples_per_class=5000)


In [ ]:
# X.shape

In [ ]:
# model = RNN(**MODELS_CONFIG["LSTM"][1])
# cv_results = cross_validate(
#             model, X, y, 
#             cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True), 
#             scoring={
#                 "accuracy": "accuracy",
#                 'f1_weighted':'f1_weighted',
#                 'recall':'recall',
#                 # "memory": forward_memory_scorer
#             },
#             n_jobs=-1
#         )
# model.fit(X, y)

# cv_results

In [ ]:
# print(model.predict(X[0:2000]), y[0:2000])
# print(y.shape[0])
# len(y[y == 1])

In [ ]:
# model.model

In [ ]:
def forward_memory_scorer(clf, x, y):
    try:
        return max(memory_usage((clf.predict, [x], {})))
    except:
        return 0



In [ ]:
metrics = ["Accuracy", "F1", "Recall", "Forward time", "Fit time"] #, "Forward memory"

def run_test(model_cls, params):
    results = {}
    for path, name in tqdm(EMBEDDING_FILES):
        # print(name)
        X, y = load_embeddings(path, max_samples_per_class=5000)
        print("loaded ", X.shape)
        model = model_cls()
        params["in_dim"] = [X.shape[-1]]
        # params["in_dim"] = X.shape[-1]
        best_params = HalvingGridSearchCV(
            model,
            params, 
            resource='n_samples',
            scoring='accuracy',
            random_state=1, 
            cv=StratifiedKFold(n_splits=3, random_state=1, shuffle=True),
            verbose=1,
            n_jobs=-1,
            refit=False,
            # aggressive_elimination=True,
            # max_resources=5000
        ).fit(X, y).best_params_
        # print(params)

        model = model_cls(**best_params)
        cv_results = cross_validate(
            model, X, y, 
            cv=StratifiedKFold(n_splits=5, random_state=2, shuffle=True), 
            scoring={
                "accuracy": "accuracy",
                'f1_weighted':'f1_weighted',
                'recall':'recall',
                # "memory": forward_memory_scorer
            },
            n_jobs=3
        )
        print(cv_results)
        results[name] = {
            "Accuracy": cv_results["test_accuracy"].tolist(),
            "F1": cv_results["test_f1_weighted"].tolist(),
            "Recall": cv_results["test_recall"].tolist(),
            # "Forward memory": cv_results["test_memory"].tolist(),
            "Forward time": cv_results["score_time"].tolist(),
            "Fit time": cv_results["fit_time"].tolist(),
            "params": params
        }
    return results

In [ ]:
MODELS_CONFIG = {
    "LSTM": (RNN, {
        "epochs": [20, 50],
        "layers": [1, 3],
        "hid_dim": [50, 100],
        # "lin_hid": [64, 128],
        "bi": [True], 
        "dropout": [0.2, 0.4]
    }),
    "Conv": (Conv, {
        "epochs": [20, 50],
        "layers": [1, 3],
        "hid_ch": [40, 60],
        "kernel_size": [2, 4],
        # "lin_hid": [64, 128],
    }),
    "MLP": (MLP, {
        "epochs": [20,  50],
        "layers": [1, 2, 3, 5],
        "hid_dim": [30, 50, 100]
    }),
}

# MODELS_CONFIG = {
#     "MLP": (MLP, {
#         "epochs": 50,
#         "layers":  3,
#         "hid_dim": 50
#     }),
#     "Conv": (Conv, {
#         "epochs": 50,
#         "layers": 1,
#         "hid_ch": 60,
#         "kernel_size": 4,
#         # "lin_hid": [64, 128],
#     }),
#     "LSTM": (RNN, {
#         "epochs": 50,
#         "layers": 1,
#         "hid_dim": 50,
#         # "lin_hid": [64, 128],
#         "bi": True, 
#         "dropout": 0.3
#     }),
    
# }

In [ ]:
# results = {name: run_test(config[0], config[1]) for name, config in MODELS_CONFIG.items()}

results = {}
# os.system(f'rm -rf {log_dir}*')
for name, config in MODELS_CONFIG.items():
    results[name] = run_test(config[0], config[1])
    with open(os.path.join(BASE_DIR, 'results_3.json'), 'w') as f:
        json.dump(results, f)

  0%|          | 0/5 [00:00<?, ?it/s]

loaded  (10000, 128, 100)
{'epochs': 50, 'layers': 3, 'hid_dim': 50, 'in_dim': 100}


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

{'fit_time': array([84.46912384, 84.81758523, 84.05219436, 84.18989205, 83.94679832]), 'score_time': array([0.05664134, 0.05437088, 0.0549953 , 0.06041694, 0.05587411]), 'test_accuracy': array([0.497 , 0.4975, 0.501 , 0.4945, 0.5   ]), 'test_f1_weighted': array([0.3346244 , 0.34000357, 0.34333465, 0.3369133 , 0.33772641]), 'test_recall': array([0.003, 0.009, 0.011, 0.007, 0.005])}


In [ ]:
with open(os.path.join(BASE_DIR, 'results_2.json'), 'r') as f:
    results = json.load(f)
print(results)

{'MLP': {'Booking CBOW': {'Accuracy': [0.896, 0.836, 0.919, 0.901, 0.907], 'F1': [0.8950121695028523, 0.8317463802388174, 0.9185540017133825, 0.9001549111681271, 0.9062907144174255], 'Recall': [0.799, 0.677, 0.845, 0.809, 0.82], 'Forward time': [0.03720211982727051, 0.030811071395874023, 0.0319216251373291, 0.03299403190612793, 0.03438377380371094], 'Fit time': [57.75106596946716, 58.19389510154724, 58.23615884780884, 58.494845151901245, 58.37687635421753], 'params': {'epochs': 50, 'layers': 3, 'hid_dim': 50, 'in_dim': 100}}, 'Booking Skip-Gram': {'Accuracy': [0.905, 0.9115, 0.913, 0.91, 0.915], 'F1': [0.9042242161508217, 0.9109652349423725, 0.9124396135265701, 0.9093604473162636, 0.9144930291699486], 'Recall': [0.815, 0.834, 0.833, 0.826, 0.838], 'Forward time': [0.03271913528442383, 0.029211759567260742, 0.02915167808532715, 0.028046846389770508, 0.03352856636047363], 'Fit time': [58.41750717163086, 58.54174208641052, 58.43543195724487, 58.208457469940186, 60.99621844291687], 'params

In [ ]:
for metric in metrics:
    fig = go.Figure(
        layout=go.Layout(
                        template='presentation+none',
                        title=dict(
                            text=f"{metric}"
                        ),
                        barmode='overlay',
                          margin=dict(b=240),
                         xaxis=dict(tickangle=90)
                    )
    )
    traces = []
    for model, res in results.items():
        for name, data in res.items():
            n = f"{name}_{model}"
            traces.append((n, go.Box(y=data[metric], name=n, boxpoints='outliers', showlegend=False)))
            # fig.add_trace(go.Box(y=data[metric], name=n, boxpoints='outliers'))
    traces.sort(key=lambda k: k[0])
    for t in traces:
        fig.add_trace(t[1])
    fig.show(renderer='colab')
    fig.write_image(os.path.join(FIGURES_DIR, f"L3_{metric}_2.png"))

In [ ]:
for model_name, res in results.items():
    headers = ["Dataset"]
    vals = [[]]
    for row_idx, (ds_name, r) in enumerate(res.items()):
        params = r['params']
        vals[0].append(ds_name)
        for col_idx, (n, v) in enumerate(params.items()):
            if len(headers) <= len(params):
                headers.append(n)
            if len(vals) <= col_idx+1:
                vals.append([])
            vals[col_idx+1].append(v)
    fig = go.Figure(
        data=[go.Table(header=dict(values=headers),cells=dict(values=vals))],
        layout=go.Layout(
                        template='presentation+none',
                        title=dict(
                            text=f"Params: {model_name}"
                        ),
                        barmode='overlay'
                    )
    )
    fig.show(renderer='colab')
    fig.write_image(os.path.join(FIGURES_DIR, f"L3_{model_name}_params_2.png"))
